## read data from files

In [1]:
import janitor
import os
import glob
#import mitosheet
import ipywidgets as widgets
%matplotlib inline
#%matplotlib widget
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters

from sklearn.model_selection import cross_val_score


%config InlineBackend.figure_format='retina'

register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 22, 10

RANDOM_SEED = 42

In [2]:
#!pip install sklearn

In [3]:
import pandas as pd
p=0.50
y=pd.read_csv("y.csv", parse_dates=["date"], index_col="date")
X=pd.read_csv("X.csv", parse_dates=["date"], index_col="date")
#y = y.values.ravel()
X_train = X[:int(X.shape[0]*p)]
X_test = X[int(X.shape[0]*p):]
y_train = y[:int(X.shape[0]*p)]
y_test = y[int(X.shape[0]*p):]
X_train.shape

(4620, 74)

In [4]:
period = "30T"
max_ = X_train.resample(period).max()
max_.columns +="_max"
min_ = X_train.resample(period).min()
min_.columns +="_min"
mean_ = X_train.resample(period).mean()
mean_.columns +="_mean"
std_ = X_train.resample(period).std()
std_.columns +="_std"
sum_ = X_train.resample(period).sum()
sum_.columns +="_sum"
dfs=[max_,min_,mean_,std_, sum_]

In [5]:
y1 = y_train.resample(period).mean()
y1 =y1.values.ravel()

In [6]:
import functools 
df = functools.reduce(lambda left,right: pd.merge(left,right,on='date'), dfs)

In [7]:
X1 =df

In [8]:
y1

array([ 699.53066667,  445.446     ,  442.748     ,  440.00733333,
        440.06066667,  438.72266667,  438.63966667,  440.32233333,
        335.499     ,  315.649     ,  318.41433333,  316.961     ,
        315.947     ,  315.452     ,  314.38366667,  314.397     ,
        317.90033333,  315.48366667,  316.857     ,  314.84233333,
        318.228     ,  317.67233333,  317.31333333,  318.041     ,
        318.005     ,  316.82633333,  318.56633333,  317.76133333,
        317.04933333,  316.52866667,  315.26933333,  315.848     ,
        316.98933333,  316.166     ,  317.32      ,  316.698     ,
        315.20033333,  316.396     ,  316.70066667,  316.16633333,
        315.07933333,  316.24266667,  315.43633333,  315.95866667,
        317.132     ,  313.213     ,  317.54666667,  316.746     ,
        317.618     ,  316.97033333,  315.458     ,  313.423     ,
        315.37066667,  313.784     ,  315.41633333,  317.614     ,
        316.75333333,  316.61166667,  319.055     ,  316.83366

In [9]:
for i in X1.index:
    print(i)

2022-03-25 16:30:00
2022-03-25 17:00:00
2022-03-25 17:30:00
2022-03-25 18:00:00
2022-03-25 18:30:00
2022-03-25 19:00:00
2022-03-25 19:30:00
2022-03-25 20:00:00
2022-03-25 20:30:00
2022-03-25 21:00:00
2022-03-25 21:30:00
2022-03-25 22:00:00
2022-03-25 22:30:00
2022-03-25 23:00:00
2022-03-25 23:30:00
2022-03-26 00:00:00
2022-03-26 00:30:00
2022-03-26 01:00:00
2022-03-26 01:30:00
2022-03-26 02:00:00
2022-03-26 02:30:00
2022-03-26 03:00:00
2022-03-26 03:30:00
2022-03-26 04:00:00
2022-03-26 04:30:00
2022-03-26 05:00:00
2022-03-26 05:30:00
2022-03-26 06:00:00
2022-03-26 06:30:00
2022-03-26 07:00:00
2022-03-26 07:30:00
2022-03-26 08:00:00
2022-03-26 08:30:00
2022-03-26 09:00:00
2022-03-26 09:30:00
2022-03-26 10:00:00
2022-03-26 10:30:00
2022-03-26 11:00:00
2022-03-26 11:30:00
2022-03-26 12:00:00
2022-03-26 12:30:00
2022-03-26 13:00:00
2022-03-26 13:30:00
2022-03-26 14:00:00
2022-03-26 14:30:00
2022-03-26 15:00:00
2022-03-26 15:30:00
2022-03-26 16:00:00
2022-03-26 16:30:00
2022-03-26 17:00:00


In [ ]:
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
estimator = SVR(kernel="linear")
selector = RFE(estimator, n_features_to_select=10, step=1)
selector = selector.fit(X1, y1)
selector.support_
selector.ranking_

In [ ]:
selector.ranking_

In [ ]:
plt.plot(y_train)
plt.show()

In [ ]:
# from sklearn.datasets import make_friedman1
# from sklearn.feature_selection import RFECV
# from sklearn.svm import SVR
# #X, y = make_friedman1(n_samples=50, n_features=10, random_state=0)
# estimator = SVR(kernel="linear")
# selector = RFECV(estimator, step=1, cv=5)
# selector = selector.fit(X1, y1)
# selector.support_

# selector.ranking_

In [ ]:
# from sklearn.datasets import make_friedman1
# from sklearn.feature_selection import RFE
# from sklearn.svm import SVR
# # X, y = make_friedman1(n_samples=50, n_features=10, random_state=0)
# estimator = SVR(kernel="linear")
# selectorRFE = RFE(estimator, n_features_to_select=10, step=1)
# selectorRFE = selectorRFE.fit(X_train, y_train)
# selectorRFE.support_
# selectorRFE.ranking_

In [ ]:
selectorRFE.feature_names_in_

In [ ]:
colonesSelected = []
for i,j in enumerate(selectorRFE.ranking_):
    # print(j)
    if j<5:
        colonesSelected.append(i)

In [ ]:
var=X_train.columns[colonesSelected]

In [ ]:
var

In [5]:
var =['temperature_103', 'light_103', 'sound_103', 'temperature_104',
       'light_104', 'light_105', 'light_106', 'temperature_107', 'light_107',
       'tvoc_108', 'sound_108', 'temperature_110', 'temperature_112']

In [6]:
import pandas
from sklearn import linear_model

X = X[var].copy()
print(X.columns)
col = [0, 4, 5, 7, 8, 9, 13, 14, 15, 17, 18, 21, 25, 26, 29, 30, 32, 33, 34, 35, 36]
col = [0, 4, 5, 7,  9, 13, 15, 18, 21, 26,  30]
col2    =[0, 1, 2, 3, 4, 5,6,  8, 10, 11, 13, 14,15]
# X= X[col]
#X= X.iloc[:,col]
y = y
# importing train_test_split from sklearn
from sklearn.model_selection import train_test_split
# splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, shuffle = False)

X_train = X_train.values
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
datascaler=scaler.fit(X_train)
X_train= datascaler.transform(X_train)
X_test= datascaler.transform(X_test)

regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)


print(regr.coef_) 

Index(['temperature_103', 'light_103', 'sound_103', 'temperature_104',
       'light_104', 'light_105', 'light_106', 'temperature_107', 'light_107',
       'tvoc_108', 'sound_108', 'temperature_110', 'temperature_112'],
      dtype='object')
[[-2.51138950e+01 -6.04513917e+01 -4.23076081e-01  8.23998451e+01
   3.36715547e+01  5.12095935e+00  6.72743693e+02  6.83095055e+01
  -5.31896690e+02  4.32990561e+01 -7.60309891e+00  3.77790832e+00
  -5.02930975e+01]]


In [7]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy import stats

X2 = sm.add_constant(X_train)
est = sm.OLS(y_train, X2)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:     elec_general_219_w   R-squared:                       0.804
Model:                            OLS   Adj. R-squared:                  0.804
Method:                 Least Squares   F-statistic:                     2331.
Date:                Fri, 06 May 2022   Prob (F-statistic):               0.00
Time:                        14:38:29   Log-Likelihood:                -46285.
No. Observations:                7392   AIC:                         9.260e+04
Df Residuals:                    7378   BIC:                         9.269e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        434.7110      1.476    294.508      0.0

In [8]:
lm = linear_model.LinearRegression()
X, y=X_train, y_train
lm.fit(X,y)
params = np.append(lm.intercept_,lm.coef_)
predictions = lm.predict(X)

newX = pd.DataFrame({"Constant":np.ones(len(X))}).join(pd.DataFrame(X))
MSE = (sum((y-predictions)**2))/(len(newX)-len(newX.columns))

# Note if you don't want to use a DataFrame replace the two lines above with
# newX = np.append(np.ones((len(X),1)), X, axis=1)
# MSE = (sum((y-predictions)**2))/(len(newX)-len(newX[0]))

var_b = MSE*(np.linalg.inv(np.dot(newX.T,newX)).diagonal())
sd_b = np.sqrt(var_b)
ts_b = params/sd_b

p_values =[2*(1-stats.t.cdf(np.abs(i),(len(newX)-len(newX.columns)-1))) for i in ts_b]

sd_b = np.round(sd_b,3)
ts_b = np.round(ts_b,3)
p_values = np.round(p_values,3)
params = np.round(params,4)

myDF3 = pd.DataFrame()
myDF3["Coefficients"],myDF3["Standard Errors"],myDF3["t values"],myDF3["Probabilities"] = [params,sd_b,ts_b,p_values]
print(myDF3)

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [ ]:
col= []
for i,pval in enumerate(myDF3["Probabilities"]):
    if pval<=0.001:
        col.append(i-1)
col   = col[1:] 

In [ ]:
col

In [9]:
# importing r2_score module
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
# predicting the accuracy score
score=r2_score(y_test,y_pred)
print("r2 socre is ",score)
print("mean_sqrd_error is ",mean_squared_error(y_test,y_pred))
print("root_mean_squared error of is ",np.sqrt(mean_squared_error(y_test,y_pred)))

r2 socre is  0.5818755142341248
mean_sqrd_error is  8373.730622540243
root_mean_squared error of is  91.50809047587127


In [10]:
d = {'y_pred': y_pred, 'y_test': y_test}
dfplot= pd.DataFrame(data=d)
dfplot.plot()


ValueError: If using all scalar values, you must pass an index

## Missing data generation

In [ ]:
#!pip install torch
#!pip install wget 
import wget
#wget.download('https://raw.githubusercontent.com/BorisMuzellec/MissingDataOT/master/utils.py')

import numpy as np
import pandas as pd
from utils import *
import torch
import seaborn as sns

In [ ]:
df1 = df.copy()



In [ ]:
df.plot()

In [ ]:
#!pip install category_encoders
import category_encoders as ce
# Get a new clean dataframe
obj_df = df1.select_dtypes(include=['object']).copy()

In [ ]:
df1 = obj_df.replace(x)

In [ ]:
df1.plot()

In [ ]:
# Fix the seed ------------------------------------------------------
np.random.seed(2)

In [ ]:
# Function produce_NA for generating missing values ------------------------------------------------------

def produce_NA(X, p_miss, mecha="MCAR", opt=None, p_obs=None, q=None):
    """
    Generate missing values for specifics missing-data mechanism and proportion of missing values. 
    
    Parameters
    ----------
    X : torch.DoubleTensor or np.ndarray, shape (n, d)
        Data for which missing values will be simulated.
        If a numpy array is provided, it will be converted to a pytorch tensor.
    p_miss : float
        Proportion of missing values to generate for variables which will have missing values.
    mecha : str, 
            Indicates the missing-data mechanism to be used. "MCAR" by default, "MAR", "MNAR" or "MNARsmask"
    opt: str, 
         For mecha = "MNAR", it indicates how the missing-data mechanism is generated: using a logistic regression ("logistic"), quantile censorship ("quantile") or logistic regression for generating a self-masked MNAR mechanism ("selfmasked").
    p_obs : float
            If mecha = "MAR", or mecha = "MNAR" with opt = "logistic" or "quanti", proportion of variables with *no* missing values that will be used for the logistic masking model.
    q : float
        If mecha = "MNAR" and opt = "quanti", quantile level at which the cuts should occur.
    
    Returns
    ----------
    A dictionnary containing:
    'X_init': the initial data matrix.
    'X_incomp': the data with the generated missing values.
    'mask': a matrix indexing the generated missing values.s
    """
    
    to_torch = torch.is_tensor(X) ## output a pytorch tensor, or a numpy array
    if not to_torch:
        X = X.astype(np.float32)
        X = torch.from_numpy(X)
    
    if mecha == "MAR":
        mask = MAR_mask(X, p_miss, p_obs).double()
    elif mecha == "MNAR" and opt == "logistic":
        mask = MNAR_mask_logistic(X, p_miss, p_obs).double()
    elif mecha == "MNAR" and opt == "quantile":
        mask = MNAR_mask_quantiles(X, p_miss, q, 1-p_obs).double()
    elif mecha == "MNAR" and opt == "selfmasked":
        mask = MNAR_self_mask_logistic(X, p_miss).double()
    else:
        mask = (torch.rand(X.shape) < p_miss).double()
    
    X_nas = X.clone()
    X_nas[mask.bool()] = np.nan
    
    return {'X_init': X.double(), 'X_incomp': X_nas.double(), 'mask': mask}

In [ ]:
X = df.copy()
shelly_219 =  ['state_AA7BF5','state_AA4706', 'state_AA9130', 'state_483FDA81E0F4']
for shelly in shelly_219:
    X = X.drop([shelly ], axis=1)

In [ ]:
X.columns

In [ ]:
X = X.values
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
datascaler=scaler.fit(X)
X= datascaler.transform(X)

In [ ]:
import pandas as pd
df= pd.DataFrame(X)
df.index=data.index

In [ ]:
df.head()

In [ ]:
#df.to_csv('dataComplet_219.csv')

## MCAR Generation

In [ ]:
# Minimal example for generating missing data ------------------------
p='5.csv'
X_miss_mcar = produce_NA(X, p_miss=0.05, mecha="MCAR")
X_mcar = X_miss_mcar['X_incomp']
R_mcar = X_miss_mcar['mask']

print("Percentage of newly generated missing values: ", (R_mcar.sum()).numpy()/np.prod(R_mcar.size())*100, " %")

df = X_mcar
df = df.numpy()
mask=R_mcar
mask = mask.numpy()

df = pd.DataFrame(data=df, columns=['Temperature', 'Humidity', 'CO2', 'Noise','Pressure'])
mask = pd.DataFrame(data=mask, columns=['Temperature', 'Humidity', 'CO2', 'Noise','Pressure'])

df['time']= data.index
df.set_index('time',inplace = True)
mask['time']= data.index
mask.set_index('time',inplace = True)
df.to_csv('MCAR'+p)
mask.to_csv('mask'+p)

In [ ]:
#df.to_csv('MCAR25.csv')
# supprimer d'abord les deux lignes dans le fichier d'export Neatatmo
index_col="time"
df=pd.read_csv('MCAR40.csv', sep=",", encoding = "ISO-8859-1", index_col=index_col)
mask=pd.read_csv('mask40.csv', sep=",", encoding = "ISO-8859-1", index_col=index_col)
mask=mask.values


In [ ]:
# Sample mar data -----------------------------------------
X_miss_mar = produce_NA(X, p_miss=0.4, mecha="MAR", p_obs=0.4)

X_mar = X_miss_mar['X_incomp']
R_mar = X_miss_mar['mask']

print("Percentage of generated missing values: ", (R_mar.sum()).numpy()/np.prod(R_mar.size())*100, " %")

ax = sns.heatmap(X_mar, mask=R_mar.numpy()==1, linewidths=0.02, linecolor='black')

df = X_mar
df = df.numpy()
mask_mar = R_mar
mask_mar = mask_mar.numpy()

df = pd.DataFrame(data=df, columns=['Temperature', 'Humidity', 'CO2', 'Noise','Pressure'])
mask_mar = pd.DataFrame(data=mask_mar, columns=['Temperature', 'Humidity', 'CO2', 'Noise','Pressure'])



In [ ]:
# df['time']= data.index
# df.set_index('time',inplace = True)
# mask_mar['time'] = data.index
# mask_mar.set_index('time',inplace = True)
# df.to_csv('MAR'+p)
# mask_mar.to_csv('mask_mar'+p)

In [ ]:
# Minimal example for generating missing data ------------------------
p='5.csv'

In [ ]:
pourcentage=5
p=